# Effect of repealing charitable deduction

This identifies beneficiaries of the charitable deduction by modeling its repeal. Both repeal from current (2017) state and TCJA state are considered on a static basis. Change to after-tax income by decile and share of after-tax income held by top 10% are calculated.

*taxcalc version: 0.14.3  |  Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis  |  Date run: 2017-12-30*

## Setup

### Imports

In [1]:
from __future__ import print_function  # Necessary only if using Python 2.7.
import taxcalc as tc
import urllib as url_lib  # On Python 3.6 use "import urllib.request as url_lib".

### Load reforms

Load from Github.

In [2]:
# Folders where reforms live.
GITHUB_BASE_URL = 'https://raw.githubusercontent.com/'

TAXCALC_GITHUB_BASE_URL = (GITHUB_BASE_URL +
                           'open-source-economics/Tax-Calculator/master/' +
                           'taxcalc/reforms/')

def read_url(url):
    return url_lib.urlopen(url).read()

def read_reform_github(reform_name):
    return read_url(GITHUB_BASE_URL + reform_name + '.json')

def read_reform_taxcalc_github(reform_name):
    return read_url(TAXCALC_GITHUB_BASE_URL + reform_name + '.json')

def create_static_params_github(reform_name):
    return tc.Calculator.read_json_param_objects(
        read_reform_github(reform_name), None)

def create_static_params_taxcalc_github(reform_name):
    return tc.Calculator.read_json_param_objects(
        read_reform_taxcalc_github(reform_name), None)

In [3]:
no_charitable_reform = create_static_params_github(
    'MaxGhenis/taxcalc-notebooks/master/charitable_repeal/NoCharitableDeduction')
tcja_reform = create_static_params_taxcalc_github(
    'TCJA_Reconciliation')
tcja_no_charitable_reform = create_static_params_github(
    'MaxGhenis/taxcalc-notebooks/master/charitable_repeal/TCJA_NoCharitable')

### Specify `Policy` objects for static analyses

In [4]:
recs = tc.Records.cps_constructor()

In [5]:
def static_baseline_calc(year):
    calc = tc.Calculator(records=recs, policy=Policy())
    calc.advance_to_year(year)
    calc.calc_all()
    return calc

In [6]:
def static_calc_from_reform(reform, year):
    pol = tc.Policy()
    pol.implement_reform(reform['policy'])
    if pol.reform_errors:
        print(pol.reform_errors)
    calc = tc.Calculator(records=recs, policy=pol)
    calc.advance_to_year(year)
    calc.calc_all()
    # Needs more if adding behavior.
    return calc

In [7]:
baseline_calc = static_baseline_calc(2018)
no_charitable_calc = static_calc_from_reform(no_charitable_reform, 2018)
tcja_calc = static_calc_from_reform(tcja_reform, 2018)
tcja_no_charitable_calc = static_calc_from_reform(tcja_no_charitable_reform, 2018)

NameError: global name 'Policy' is not defined

## Analysis

### Change to aggregate individual income tax revenue

In [ ]:
print ('Pre-TCJA: ${:0.1f}B'.
       format((no_charitable_calc.weighted_total('iitax') - 
               baseline_calc.weighted_total('iitax')) / 1e9))
print ('TCJA: ${:0.1f}B'.
       format((tcja_no_charitable_calc.weighted_total('iitax') - 
               tcja_calc.weighted_total('iitax')) / 1e9))

### Construct difference tables by income decile

Ignore errors ([issue](https://github.com/open-source-economics/Tax-Calculator/issues/1799)).

In [ ]:
baseline_diff_table = baseline_calc.difference_table(no_charitable_calc, tax_to_diff='iitax')
tcja_diff_table = tcja_calc.difference_table(tcja_no_charitable_calc, tax_to_diff='iitax')

In [ ]:
baseline_diff_table

In [ ]:
tcja_diff_table

Focus on % change to after-tax income.

In [ ]:
pd.DataFrame({'baseline': -baseline_diff_table['pc_aftertaxinc'],
              'tcja': -tcja_diff_table['pc_aftertaxinc']})

Graphed in Google Sheets [here](https://docs.google.com/spreadsheets/d/17ozKKuWUzgI4yn1OwqshmPBVWVklQ1GlpTAUepuwsF8/edit?usp=sharing).

*TODO: Make a Seaborn or Bokeh plot here instead.*

### After-tax income held by top decile

Ignore errors ([issue](https://github.com/open-source-economics/Tax-Calculator/issues/1799)).

In [ ]:
baseline_dist, no_charitable_dist = baseline_calc.distribution_tables(no_charitable_calc)
tcja_dist, tcja_no_charitable_dist = tcja_calc.distribution_tables(tcja_no_charitable_calc)

In [ ]:
def after_tax_income_top10pct(calc):
    return calc.aftertax_income[9] / calc.aftertax_income[10]

In [ ]:
print ('After-tax income from top 10% - Baseline: {:0.3f}%'.
       format(after_tax_income_top10pct(baseline_dist) * 100))
print ('After-tax income from top 10% - No charitable: {:0.3f}%'.
       format(after_tax_income_top10pct(no_charitable_dist) * 100))
print ('After-tax income from top 10% - Difference: {:0.3f}%'.
       format((after_tax_income_top10pct(no_charitable_dist) - 
               after_tax_income_top10pct(baseline_dist)) * 100))

In [ ]:
print ('After-tax income from top 10% - Baseline: {:0.3f}%'.
       format(after_tax_income_top10pct(tcja_dist) * 100))
print ('After-tax income from top 10% - No charitable: {:0.3f}%'.
       format(after_tax_income_top10pct(tcja_no_charitable_dist) * 100))
print ('After-tax income from top 10% - Difference: {:0.3f}%'.
       format((after_tax_income_top10pct(tcja_no_charitable_dist) - 
               after_tax_income_top10pct(tcja_dist)) * 100))